# Rastreamento de um objeto utilizando o método SIFT

In [3]:
import numpy as np
import cv2
import tracking

class RegionSelector:
    """Classe para capturar o desenho de um retângulo na tela. Utilizada para 
    capturar a posição inicial de um objeto."""
    
    def __init__(self, img):

        self.img = img
        self.img2draw = img
        self.drawing = False
        self.initial_x = -1
        self.initial_y = -1
        self.end_selection = False

        cv2.namedWindow('image')
        cv2.setMouseCallback('image', self.draw_rectangle)

        while not self.end_selection:
            cv2.imshow('image', self.img2draw)
            if cv2.waitKey(1) & 0xFF == 27:
                break
        cv2.destroyAllWindows()

    def draw_rectangle(self, event, x, y, flags, param):

        if event == cv2.EVENT_LBUTTONDOWN:
            self.drawing = True
            self.initial_x = x
            self.initial_y = y

        elif event == cv2.EVENT_MOUSEMOVE:
            if self.drawing == True:
                ix = self.initial_x
                iy = self.initial_y
                self.img2draw = self.img.copy()
                cv2.rectangle(self.img2draw,(ix,iy),(x,y),(0,255,0),2)

        elif event == cv2.EVENT_LBUTTONUP:
            self.drawing = False
            ix = self.initial_x
            iy = self.initial_y
            self.img2draw = self.img.copy()
            cv2.rectangle(self.img2draw,(ix,iy),(x,y),(0,255,0),2)

            upper_left_x = ix if ix<x else x
            upper_left_y = iy if iy < y else y
            w = np.abs(ix - x)
            h = np.abs(iy - y)
            self.track_window = (upper_left_x, upper_left_y, w, h)
            self.end_selection = True

class Tracker:
    """Rastreia um objeto selecionado pelo usuário em cada quadro de uma câmera
    ligada ao computador."""

    def __init__(self):

        # Inicializa câmera
        self.cap = cv2.VideoCapture(1)
        # Captura imagem do objeto a ser rastreado
        img_obj_g = self.select_object()

        # Pontos salientes do objeto, calculados apenas uma vez
        sift = cv2.SIFT_create(
            nfeatures=0, nOctaveLayers=3, contrastThreshold=0.02, 
            edgeThreshold=10, sigma=1.6
            )
        kp_obj, des_obj = sift.detectAndCompute(img_obj_g, None)

        self.img_obj_g = img_obj_g
        self.sift = sift
        self.kp_obj = kp_obj
        self.des_obj = des_obj

        self.start_tracking()

    def select_object(self):
        """Permite que o usuário capture um quadro da câmera com a tecla 'c'. 
        Na sequência, o usuário desenha um retângulo no quadro capturado para
        indicar o objeto a ser rastreado."""

        # Captura a imagem atual da câmera quando o usuário pressiona a tecla 'q'
        # A imagem capturada será usada para selecionar o objeto a ser rastreado
        while(True):
            ret, frame = self.cap.read()
            cv2.imshow('frame', frame)
            if cv2.waitKey(1) & 0xFF == ord('c'):
                break

        # Seleção do objeto pelo usuário
        rs = RegionSelector(frame)        
        upper_left_x, upper_left_y, w, h = rs.track_window

        img_obj = frame[upper_left_y:upper_left_y+h, upper_left_x:upper_left_x+w]
        img_obj_g = cv2.cvtColor(img_obj, cv2.COLOR_BGR2GRAY)

        return img_obj_g
    
    def start_tracking(self):
        """Detecta o objeto em cada quadro de vídeo até que o usuário pressione
        a tecla 'q'"""

        while(True):
            # Captura imagem atual da câmera
            ret, frame = self.cap.read()
            # Redimensiona imagem para melhor performance
            frame_res = cv2.resize(frame, (640, 360))
            frame_g_res = cv2.cvtColor(frame_res, cv2.COLOR_BGR2GRAY)
            
            # Encontra posição do objeto
            obj_bounds_in_scene = tracking.find_object(
                frame_g_res, 
                self.img_obj_g,
                self.kp_obj, 
                self.des_obj, 
                self.sift
                )  
            # Desenha contorno do objeto
            img_bbox = tracking.draw_bbox(frame_res, obj_bounds_in_scene)

            # Mostra o resultado
            cv2.imshow('frame', img_bbox)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        self.cap.release()
        cv2.destroyAllWindows()
                
Tracker()